In [1]:
import pandas as pd
import numpy as np
import sqlite3


In [2]:
np.random.seed(42)
n = 1000

categories = ['Fashion', 'Electronics', 'Home', 'Beauty', 'Footwear']
data = {
    "SKU": np.random.randint(1000, 1100, size=n),
    "Category": np.random.choice(categories, size=n),
    "Sales": np.round(np.random.uniform(500, 5000, size=n), 2),
    "Returns": np.round(np.random.uniform(0, 1000, size=n), 2),
    "COGS": np.round(np.random.uniform(200, 2500, size=n), 2),  # Cost of goods sold
    "LogisticsCost": np.round(np.random.uniform(50, 300, size=n), 2),
    "MarketingCost": np.round(np.random.uniform(30, 200, size=n), 2)
}
df = pd.DataFrame(data)
df.head()


,SKU,Category,Sales,Returns,COGS,LogisticsCost,MarketingCost
0,1051,Electronics,3477.35,709.01,1260.68,54.17,67.42
1,1092,Fashion,4903.84,177.17,2361.84,98.69,73.18
2,1014,Beauty,2184.34,507.45,1359.21,252.62,179.21
3,1071,Home,578.82,677.39,2383.16,118.16,45.23
4,1060,Electronics,1598.66,235.04,2115.51,232.89,151.61


In [3]:
# Group by Category and aggregate sums
category_pnl = df.groupby('Category').agg({
    'Sales': 'sum',
    'Returns': 'sum',
    'COGS': 'sum',
    'LogisticsCost': 'sum',
    'MarketingCost': 'sum'
}).reset_index()

# Calculate Net Revenue, Total Cost, Net Profit, Profit Margin
category_pnl['NetRevenue'] = category_pnl['Sales'] - category_pnl['Returns']
category_pnl['TotalCost'] = category_pnl['COGS'] + category_pnl['LogisticsCost'] + category_pnl['MarketingCost']
category_pnl['NetProfit'] = category_pnl['NetRevenue'] - category_pnl['TotalCost']
category_pnl['ProfitMargin'] = (category_pnl['NetProfit'] / category_pnl['NetRevenue']) * 100

category_pnl = category_pnl.round(2)
category_pnl


,Category,Sales,Returns,COGS,LogisticsCost,MarketingCost,NetRevenue,TotalCost,NetProfit,ProfitMargin
0,Beauty,528969.76,90596.08,265683.15,33823.92,21963.20,438373.68,321470.27,116903.41,26.67
1,Electronics,541648.49,106969.98,263577.86,33523.59,23136.82,434678.51,320238.27,114440.24,26.33
2,Fashion,558374.29,104786.88,270268.70,33671.88,23004.52,453587.41,326945.10,126642.31,27.92
3,Footwear,541717.68,108913.79,274215.42,36247.72,22933.05,432803.89,333396.19,99407.70,22.97
4,Home,556421.69,106141.38,261598.52,34846.94,22660.45,450280.31,319105.91,131174.40,29.13


In [4]:
conn = sqlite3.connect("category_pnl.db")
df.to_sql("sku_data", conn, if_exists="replace", index=False)
category_pnl.to_sql("category_pnl", conn, if_exists="replace", index=False)


5

In [5]:
# Top 3 categories by Net Profit
query1 = pd.read_sql_query("""
    SELECT Category, NetProfit
    FROM category_pnl
    ORDER BY NetProfit DESC
    LIMIT 3
""", conn)
print("Top 3 Categories by Net Profit:\n", query1)

# Categories with Negative Profit Margin
query2 = pd.read_sql_query("""
    SELECT Category, NetProfit, ProfitMargin
    FROM category_pnl
    WHERE ProfitMargin < 0
""", conn)
print("\nCategories with Negative Profit Margin:\n", query2)

# Category-wise summary for dashboarding
query3 = pd.read_sql_query("""
    SELECT Category, NetRevenue, TotalCost, NetProfit, ProfitMargin
    FROM category_pnl
    ORDER BY NetProfit ASC
""", conn)
print("\nCategory-wise P&L Summary:\n", query3)


Top 3 Categories by Net Profit:
   Category  NetProfit
0     Home  131174.40
1  Fashion  126642.31
2   Beauty  116903.41

Categories with Negative Profit Margin:
 Empty DataFrame
Columns: [Category, NetProfit, ProfitMargin]
Index: []

Category-wise P&L Summary:
       Category  NetRevenue  TotalCost  NetProfit  ProfitMargin
0     Footwear   432803.89  333396.19   99407.70         22.97
1  Electronics   434678.51  320238.27  114440.24         26.33
2       Beauty   438373.68  321470.27  116903.41         26.67
3      Fashion   453587.41  326945.10  126642.31         27.92
4         Home   450280.31  319105.91  131174.40         29.13


In [7]:
category_pnl.to_csv("category_pnl_summary.csv", index=False)
print("Exported category-level P&L summary as 'category_pnl_summary.csv'")


Exported category-level P&L summary as 'category_pnl_summary.csv'


 Business Recommendations (Markdown Cell)
Sample Recommendations:

Focus on high-profit categories for marketing and assortment expansion.

Investigate and improve operational efficiency in categories with negative profit margin.

Revisit pricing or return policy for high-loss categories.

Use exported CSV for Power BI/Excel dashboards to monitor category health.

In [8]:
conn.close()
